## Scenario 2: A cross-functional team with one data scientist working on an ML model

`MLflow setup:`

- Tracking server:  *yes, local server*
- Backend store:  *sqlite database*
- Artifacts store:  *local filesystem*

The experiments can be explored locally by accessing the local tracking server. However, the metadata of the experiments are to be stored in the backend sql database for collaboration.

Here we need to launch the mlflow server locally by running the following command in your terminal:

        `mlflow server --backend-store-uri sqlite:///backend.db`

In [1]:
import mlflow

In [2]:
MLFLOW_TRACKING_URI = "http://127.0.0.1:5000"

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [3]:
from mlflow.entities import ViewType

mlflow.search_experiments(view_type = ViewType.ALL)

[<Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1685367771404, experiment_id='0', last_update_time=1685367771404, lifecycle_stage='active', name='Default', tags={}>]

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run(experiment_id= '1'):

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")


2023/05/29 19:16:06 INFO mlflow.tracking.fluent: Experiment with name 'my-experiment-1' does not exist. Creating a new experiment.
/Users/sahelimukherjee/Documents/Personal/Learning/MLOps/projects/venv/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [5]:
mlflow.search_experiments(view_type = ViewType.ALL)

[<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1685367967218, experiment_id='1', last_update_time=1685367967218, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1685367771404, experiment_id='0', last_update_time=1685367771404, lifecycle_stage='active', name='Default', tags={}>]

## Interacting with the model registry

In [6]:
from mlflow.tracking import MlflowClient

client = MlflowClient(MLFLOW_TRACKING_URI)

In [7]:
f"runs:/{client.search_runs('1')[0].info.run_id}/model"

'runs:/19250c5bf19643b6a6757617f98abc77/model'

In [8]:
## register model
run_id = f"runs:/{client.search_runs('1')[0].info.run_id}/models"
model_name = "iris-classifier"

mlflow.register_model(model_uri= run_id,
                      name = model_name)

Successfully registered model 'iris-classifier'.
2023/05/29 19:18:06 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1685368086369, current_stage='None', description='', last_updated_timestamp=1685368086369, name='iris-classifier', run_id='19250c5bf19643b6a6757617f98abc77', run_link='', source='mlflow-artifacts:/1/19250c5bf19643b6a6757617f98abc77/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>

In [9]:
## list registered models
client.search_registered_models()

[<RegisteredModel: aliases={}, creation_timestamp=1685368086332, description='', last_updated_timestamp=1685368086369, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1685368086369, current_stage='None', description='', last_updated_timestamp=1685368086369, name='iris-classifier', run_id='19250c5bf19643b6a6757617f98abc77', run_link='', source='mlflow-artifacts:/1/19250c5bf19643b6a6757617f98abc77/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>], name='iris-classifier', tags={}>]

In [10]:
## transition model
client.transition_model_version_stage(name = model_name,
                                      version = '1',
                                      stage = "staging",
                                      archive_existing_versions= True)

<ModelVersion: aliases=[], creation_timestamp=1685368086369, current_stage='Staging', description='', last_updated_timestamp=1685368133170, name='iris-classifier', run_id='19250c5bf19643b6a6757617f98abc77', run_link='', source='mlflow-artifacts:/1/19250c5bf19643b6a6757617f98abc77/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>

In [11]:
## load staged model for prediction
stage = "Staging"
staged_model = mlflow.pyfunc.load_model(f"models:/{model_name}/{stage}")

## make predictions and evaluate 
y_pred = staged_model.predict(X)
print(f"Accuracy : {accuracy_score(y, y_pred)}")

2023/05/29 19:19:02 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.3.2, required: mlflow==2.3)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


Accuracy : 0.96
